In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy.stats import shapiro
import os
from scipy.stats import linregress

# Reading PCB sheets

In [ ]:
sheets = ("blank_corrected", "lipid_normalized")
sheet = sheets[0]
pcb = pd.read_excel("./generated excels/pcb.xlsx", sheet_name=sheet)
c = pcb.columns.tolist()
c[0] = 'Units'
pcb.columns = c
pcb.set_index('Units', inplace=True)
selection = [i for i in pcb.index if i[:3] == 'PCB']
pcb = pcb.loc[selection, :]
pcb

# Shapiro

In [ ]:
shapi = pd.DataFrame(pcb.iloc[:, :2].copy())
shapi.columns = ['statistic', 'pvalue']
shapi
for i in pcb.index:
    shapi.loc[i, :] = shapiro(pcb.loc[i, :])
    if sum(pcb.loc[i, :].dropna().to_numpy() > 0) >= 3:
        shapi.loc[i, :] = shapiro(pcb.loc[i, :].dropna().to_numpy())

In [ ]:
shapi2 = pd.DataFrame(index = ['statistics', 'pvalue'], columns = pcb.columns, dtype='double')
shapi2
for i in pcb.columns:
    shapi2.loc[:, i] = shapiro(pcb.loc[:, i].dropna().to_numpy())
shapi2

## repeating shapiro for each pcb sheet

In [ ]:
sheets = ("blank_corrected", "lipid_normalized")
shapiro_dict = {}
for sheet in sheets:
    pcb = pd.read_excel("./generated excels/pcb.xlsx", sheet_name=sheet)
    c = pcb.columns.tolist()
    c[0] = 'Units'
    pcb.columns = c
    pcb.set_index('Units', inplace=True)
    selection = [i for i in pcb.index if i[:3] == 'PCB']
    pcb = pcb.loc[selection, :]

    shapi = pd.DataFrame(pcb.iloc[:, :2].copy())
    shapi.columns = ['statistic', 'pvalue']
    shapi
    for i in pcb.index:
        shapi.loc[i, :] = shapiro(pcb.loc[i, :])
        if sum(pcb.loc[i, :].dropna().to_numpy() > 0) >= 3:
            shapi.loc[i, :] = shapiro(pcb.loc[i, :].dropna().to_numpy())

    shapi2 = pd.DataFrame(index = ['statistics', 'pvalue'], columns = pcb.columns, dtype='double')
    shapi2
    for i in pcb.columns:
        shapi2.loc[:, i] = shapiro(pcb.loc[:, i].dropna().to_numpy())

    shapiro_dict.update({sheet: (shapi, shapi2)})

In [ ]:
sheets = ("blank_corrected", "lipid_normalized")
shapiro_log = {}
for sheet in sheets:
    pcb = pd.read_excel("./generated excels/pcb.xlsx", sheet_name=sheet)
    c = pcb.columns.tolist()
    c[0] = 'Units'
    pcb.columns = c
    pcb.set_index('Units', inplace=True)
    selection = [i for i in pcb.index if i[:3] == 'PCB']
    pcb = np.log10(pcb.loc[selection, :])

    shapi = pd.DataFrame(pcb.iloc[:, :2].copy())
    shapi.columns = ['statistic', 'pvalue']
    shapi
    for i in pcb.index:
        shapi.loc[i, :] = shapiro(pcb.loc[i, :])
        if sum(pcb.loc[i, :].dropna().to_numpy() > 0) >= 3:
            shapi.loc[i, :] = shapiro(pcb.loc[i, :].dropna().to_numpy())

    shapi2 = pd.DataFrame(index = ['statistics', 'pvalue'], columns = pcb.columns, dtype='double')
    shapi2
    for i in pcb.columns:
        shapi2.loc[:, i] = shapiro(pcb.loc[:, i].dropna().to_numpy())

    shapiro_log.update({sheet: (shapi, shapi2)})

In [ ]:
with pd.ExcelWriter('pcb_stats.xlsx') as writer:
    for sheet in sheets:
        for i in range(2):
            shapiro_dict[sheet][i].to_excel(writer, f'shapiro{i} - {sheet}')
            shapiro_log[sheet][i].to_excel(writer, f'shapiro{i} - {sheet}')

# Proportion Plots

In [ ]:
sheets = ("blank_corrected", "lipid_normalized")
sheet = sheets[1]
pcb = pd.read_excel("./generated excels/pcb.xlsx", sheet_name=sheet)
c = pcb.columns.tolist()
c[0] = 'Units'
pcb.columns = c
pcb.set_index('Units', inplace=True)
selection = [i for i in pcb.index if i[:3] == 'PCB']
pcb = pcb.loc[selection, :]
for col in pcb.columns:
    temp = pcb[col]
    temp = temp/temp.sum()*100
    plt.figure(figsize=(33, 4), facecolor="white")
    plt.bar(range(len(pcb.index)), temp.to_numpy(), .42)
    plt.title('Proportion Plot #' + col)
    plt.xticks(range(len(pcb.index)), pcb.index.str.replace('PCB-', ''), rotation = 90)
    plt.xlim(-1, len(pcb.index))
    plt.ylabel('percent(%)')
    plt.xlabel('$PCB_x$')
    if not os.path.exists('./pics/PCBs/bars/'):
        os.mkdir('./pics/PCBs/bars/')
    plt.savefig(f'./pics/PCBs/bars/{col}-proportion.jpg', bbox_inches='tight')

In [ ]:
sheets = ("blank_corrected", "lipid_normalized")
sheet = sheets[1]
pcb = pd.read_excel("./generated excels/pcb.xlsx", sheet_name=sheet)
c = pcb.columns.tolist()
c[0] = 'Units'
pcb.columns = c
pcb.set_index('Units', inplace=True)
selection = [i for i in pcb.index if i[:3] == 'PCB']
pcb = pcb.loc[selection, :]
temp = pcb.sum(1)
temp = temp/temp.sum()
plt.figure(figsize=(33, 4), facecolor="white")
plt.bar(range(len(pcb.index)), temp.to_numpy(), .42)
plt.title('Sum-Proportion')
plt.xticks(range(len(pcb.index)), pcb.index.str.replace('PCB-', ''), rotation = 90)
plt.xlim(-1, len(pcb.index))
plt.ylabel('percent(%)')
plt.xlabel('$PCB_x$')
if not os.path.exists('./pics/PCBs/bars/'):
    os.mkdir('./pics/PCBs/bars/')
plt.savefig(f'./pics/PCBs/bars/SUM-proportion.jpg', bbox_inches='tight')

temp = pcb.apply(lambda x: np.average(x.dropna()) if not x.dropna().empty else 0, 1)
temp = temp/temp.sum()
plt.figure(figsize=(33, 4), facecolor="white")
plt.bar(range(len(pcb.index)), temp.to_numpy(), .42)
plt.title('Average-Proportion')
plt.xticks(range(len(pcb.index)), pcb.index.str.replace('PCB-', ''), rotation = 90)
plt.xlim(-1, len(pcb.index))
plt.ylabel('percent(%)')
plt.xlabel('$PCB_x$')
plt.savefig(f'./pics/PCBs/bars/average-proportion.jpg', bbox_inches='tight')

# Concentration Plots

In [ ]:
sheets = ("blank_corrected", "lipid_normalized")
sheet = sheets[1]
pcb = pd.read_excel("./generated excels/pcb.xlsx", sheet_name=sheet)
c = pcb.columns.tolist()
c[0] = 'Units'
pcb.columns = c
pcb.set_index('Units', inplace=True)
selection = [i for i in pcb.index if i[:3] != 'PCB']
pcb = pcb.loc[selection, :]
pcb = pcb.iloc[:-2, :]
plt.figure(figsize=(45, 30), facecolor="white")
for i in range(len(pcb.columns)):
    if i != (len(pcb.columns)-1):
        plt.subplot(2,4,i+1)
        col = pcb.columns[i]
        temp = pcb[col]
        temp = temp/temp.sum()*100
        plt.bar(range(len(pcb.index)), temp.to_numpy(), .42)
        plt.title('Concentration Plot #' + col, fontsize = 25)
        plt.xticks(range(len(pcb.index)), pcb.index.str.removeprefix('Total '), rotation = 45, ha='right', fontweight = 'bold')
        plt.yticks(fontsize = 25)
        plt.xlim(-1, len(pcb.index))
        plt.ylabel('percent(%)', fontsize = 25)
    else:
        plt.figure(figsize = (6, 8), facecolor = 'white')
        col = pcb.columns[i]
        temp = pcb[col]
        temp = temp/temp.sum()*100
        plt.bar(range(len(pcb.index)), temp.to_numpy(), .42)
        plt.title('Concentration Plot #' + col, fontsize = 25)
        plt.xticks(range(len(pcb.index)), pcb.index, rotation = 45, ha='right')
        plt.xlim(-1, len(pcb.index))
        plt.ylabel('percent(%)')
    # plt.xlabel('$PCB_x$')
    # if not os.path.exists('./pics/PCBs/bars/'):
    #     os.mkdir('./pics/PCBs/bars/')
    # plt.savefig(f'./pics/PCBs/bars/{col}-concentration.jpg', bbox_inches='tight')

In [ ]:
sheets = ("blank_corrected", "lipid_normalized")
sheet = sheets[1]
pcb = pd.read_excel("./generated excels/pcb.xlsx", sheet_name=sheet)
c = pcb.columns.tolist()
c[0] = 'Units'
pcb.columns = c
pcb.set_index('Units', inplace=True)
selection = [i for i in pcb.index if i[:3] != 'PCB']
pcb = pcb.loc[selection, :]
pcb = pcb.iloc[:-2, :]

# Create subplots with shared y-axis
fig, axes = plt.subplots(4, 2, figsize=(12, 12), facecolor="white", sharey=True, sharex=True)
fig.supylabel("Percentage (%)", fontsize = 25)
plt.suptitle('Polychlorinated Biphenyl Congener and Homolog Patterns', x=0.125, y=0.92, horizontalalignment='left', verticalalignment='top', fontsize = 20)
for i, ax in enumerate(axes.flatten()):
    col = pcb.columns[i]
    temp = pcb[col]
    temp = temp / temp.sum() * 100
    if 3 < int(col) < 8:
        ax.bar(range(len(pcb.index)), temp.to_numpy(), .42, color = 'lightskyblue')
    else:
        ax.bar(range(len(pcb.index)), temp.to_numpy(), .42, color = 'k')
    plt.yticks(fontsize = 25)
    ax.set_title('', fontsize=25)
    ax.set_xticks(range(len(pcb.index)))
    ax.set_xticklabels(pcb.index.str.removeprefix('Total ').str.removesuffix('Biphenyls').str.removesuffix('Biphenyl'), rotation=80, ha = 'center', fontsize = 20)

    ax.set_xlim(-1, len(pcb.index))
    #ax.set_ylabel('percent(%)', fontsize=25)
    # Annotate the title on the right corner
    title_text = f'{col}'
    ax.annotate(title_text, xy=(1, max(ax.get_ylim())* 0.9), xytext=(8, 0),
                textcoords='offset points', ha='right', va='top',
                bbox=dict(boxstyle='circle,pad=0.5', fc='white', alpha=0.5),fontsize = 20)
fig, ax = plt.subplots(figsize=(12, 12), facecolor="white")
fig.supylabel("Percentage (%)", fontsize = 25)
i = i + 1
col = pcb.columns[i]
temp = pcb[col]
temp = temp / temp.sum() * 100
ax.bar(range(len(pcb.index)), temp.to_numpy(), .42, color = 'seagreen')
ax.set_title('', fontsize=25)
ax.set_xticks(range(len(pcb.index)))
ax.set_xticklabels(pcb.index.str.removeprefix('Total ').str.removesuffix('Biphenyls').str.removesuffix('Biphenyl'), rotation=80,ha = 'center', fontsize = 20)

ax.set_xlim(-1, len(pcb.index))
#ax.set_ylabel('percent(%)', fontsize=25)
# Annotate the title on the right corner
title_text = f'{col[:2]}'
ax.annotate(title_text, xy=(1, max(ax.get_ylim())* 0.9), xytext=(8, 0),
            textcoords='offset points', ha='right', va='top',
            bbox=dict(boxstyle='circle,pad=0.5', fc='white', alpha=0.5),fontsize = 30)
plt.yticks(fontsize = 20)
# Remove the title
plt.title('Polychlorinated Biphenyl Congener and Homolog Patterns', loc = 'left', fontsize = 20)

# Adjust layout
plt.tight_layout()

# Create legend outside the axis
#plt.legend(pcb.columns, fontsize=15, bbox_to_anchor=(1.05, 1), loc='upper left')

# Show plot

plt.show()


In [ ]:
sheets = ("blank_corrected", "lipid_normalized")
sheet = sheets[1]
pcb = pd.read_excel("./generated excels/pcb.xlsx", sheet_name=sheet)
c = pcb.columns.tolist()
c[0] = 'Units'
pcb.columns = c
pcb.set_index('Units', inplace=True)
selection = [i for i in pcb.index if i[:3] != 'PCB']
pcb = pcb.loc[selection, :]
pcb = pcb.iloc[:-2, :]

temp = pcb.sum(1)
temp = temp/temp.sum()
plt.figure(figsize=(10, 5), facecolor="white")
plt.bar(range(len(pcb.index)), temp.to_numpy(), .42)
plt.title('Sum-Proportion')
plt.xticks(range(len(pcb.index)), pcb.index.str.replace('PCB-', ''), rotation = 45, ha = 'right')
plt.xlim(-1, len(pcb.index))
plt.ylabel('percent(%)')
plt.xlabel('$PCB_x$')
if not os.path.exists('./pics/PCBs/bars/'):
    os.mkdir('./pics/PCBs/bars/')
plt.savefig(f'./pics/PCBs/bars/SUM-concentration.jpg', bbox_inches='tight')

temp = pcb.apply(lambda x: np.average(x.dropna()) if not x.dropna().empty else 0, 1)
temp = temp/temp.sum()
plt.figure(figsize=(10, 5), facecolor="white")
plt.bar(range(len(pcb.index)), temp.to_numpy(), .42)
plt.title('Average-Proportion')
plt.xticks(range(len(pcb.index)), pcb.index.str.replace('PCB-', ''), rotation = 45, ha = 'right')
plt.xlim(-1, len(pcb.index))
plt.ylabel('percent(%)')
plt.xlabel('$PCB_x$')
plt.savefig(f'./pics/PCBs/bars/average-concentration.jpg', bbox_inches='tight')

In [ ]:
fish_info = pd.read_excel("fish_info.xlsx")
fish_info['Fish ID'] = fish_info['Fish ID'].astype('str')
fish_info['Carcass Status'] = fish_info['Carcass Status'].astype('int8')
sheets = ("blank_corrected", "lipid_normalized")
fig, axs = plt.subplots(2, 2, figsize = (12, 10), facecolor = 'white')
for i, ax in enumerate(axs.flatten()):
    sheet = sheets[i//2]
    pcb = pd.read_excel("./generated excels/pcb.xlsx", sheet_name=sheet)
    c = pcb.columns.tolist()
    c[0] = 'Units'
    pcb.columns = c
    total_pcb = pd.DataFrame(pcb.iloc[-2, :]).reset_index()
    total_pcb.columns = total_pcb.iloc[0, :]
    total_pcb.drop(index = 0, inplace=True)
    total_pcb.iat[-1, 0] = '13'
    total_pcb.columns = ['Fish ID', 'Total PCBs']
    data = pd.merge(fish_info, total_pcb, how='inner', on='Fish ID')
    if i % 2 == 0:
        ax.scatter(*data.loc[:, ['Carcass Status', 'Total PCBs']].to_numpy().T)
        ax.set_title(sheet)
        if sheet == "blank_corrected":  
            ax.set_ylabel('wwt')
        else:
            ax.set_ylabel('LW')
    else:
        data['Total PCBs'] = np.log10(data['Total PCBs'].to_numpy('float'))
        ax.scatter(*data.loc[:, ['Carcass Status', 'Total PCBs']].to_numpy().T)
        ax.set_title('logscale ' + sheet)
        if sheet == "blank_corrected":  
            ax.set_ylabel('log10 wwt')
        else:
            ax.set_ylabel('log10 LW')
    
    ax.set_xlabel('Carcass Status')
    

fig.savefig('./pics/pcbs/vsCarcass_status.jpg')

In [ ]:
fish_info = pd.read_excel("fish_info.xlsx")
fish_info['Fish ID'] = fish_info['Fish ID'].astype('str')
fish_info['Carcass Status'] = fish_info['Carcass Status'].astype('int8')
sheets = ("blank_corrected", "lipid_normalized")
for i in range(4):
    fig, ax = plt.subplots(figsize = (12, 10), facecolor = 'white')
    sheet = sheets[i//2]
    pcb = pd.read_excel("./generated excels/pcb.xlsx", sheet_name=sheet)
    c = pcb.columns.tolist()
    c[0] = 'Units'
    pcb.columns = c
    total_pcb = pd.DataFrame(pcb.iloc[-2, :]).reset_index()
    total_pcb.columns = total_pcb.iloc[0, :]
    total_pcb.drop(index = 0, inplace=True)
    total_pcb.iat[-1, 0] = '13'
    total_pcb.columns = ['Fish ID', 'Total PCBs']
    data = pd.merge(fish_info, total_pcb, how='inner', on='Fish ID')
    if i % 2 == 0:
        ax.scatter(*data.loc[:, ['Carcass Status', 'Total PCBs']].to_numpy().T)
        ax.set_title(sheet)
        if sheet == "blank_corrected":  
            ax.set_ylabel('wwt')
        else:
            ax.set_ylabel('LW')
    else:
        data['Total PCBs'] = np.log10(data['Total PCBs'].to_numpy('float'))
        ax.scatter(*data.loc[:, ['Carcass Status', 'Total PCBs']].to_numpy().T)
        ax.set_title('logscale ' + sheet)
        if sheet == "blank_corrected":  
            ax.set_ylabel('log10 wwt')
        else:
            ax.set_ylabel('log10 LW')
    
    ax.set_xlabel('Carcass Status')
    

    fig.savefig(f'./pics/pcbs/vsCarcass_status{i}.jpg')

In [ ]:
fish_info = pd.read_excel("fish_info.xlsx")
fish_info['Fish ID'] = fish_info['Fish ID'].astype('str')
fish_info['Carcass Status'] = fish_info['Carcass Status'].astype('int8')
sheets = ("blank_corrected", "lipid_normalized")
fig, axs = plt.subplots(2, 2, figsize = (12, 10), facecolor = 'white')
for i, ax in enumerate(axs.flatten()):
    sheet = sheets[i//2]
    pcb = pd.read_excel("./generated excels/pcb.xlsx", sheet_name=sheet)
    c = pcb.columns.tolist()
    c[0] = 'Units'
    pcb.columns = c
    total_pcb = pd.DataFrame(pcb.iloc[-2, :]).reset_index()
    total_pcb.columns = total_pcb.iloc[0, :]
    total_pcb.drop(index = 0, inplace=True)
    total_pcb.iat[-1, 0] = '13'
    total_pcb.columns = ['Fish ID', 'Total PCBs']
    data = pd.merge(fish_info, total_pcb, how='inner', on='Fish ID')
    if i % 2 == 0:
        ax.scatter(*data.loc[:, ['Carcass Status', 'Total PCBs']].to_numpy().T,label = 'points', color = 'k')
        slope, intercept, r_value, p_value, std_err = linregress(*data.loc[:, ['Carcass Status', 'Total PCBs']].to_numpy('float').T)
        x = data.loc[:,'Carcass Status'].to_numpy('float')
        ax.plot(x,slope * x + intercept, label = 'linereg', color = 'green')
        ax.annotate(f'y = {slope:.2f} * x + {intercept:.2f}'+
                    "\n"+f'p_value:{p_value:.2f}' + "\n"+f'r_squared:{r_value ** 2:.2f}',
                    (2, -0.2*np.diff(ax.get_ylim())[0]+ax.get_ylim()[1]))
        ax.set_title(sheet)
        if sheet == "blank_corrected":  
            ax.set_ylabel('wwt')
        else:
            ax.set_ylabel('LW')
    else:
        data['Total PCBs'] = np.log10(data['Total PCBs'].to_numpy('float'))
        ax.scatter(*data.loc[:, ['Carcass Status', 'Total PCBs']].to_numpy().T,label = 'points', color = 'k')
        slope, intercept, r_value, p_value, std_err = linregress(*data.loc[:, ['Carcass Status', 'Total PCBs']].to_numpy('float').T)
        x = data.loc[:,'Carcass Status'].to_numpy('float')
        ax.plot(x,slope * x + intercept, label = 'linereg', color = 'green')
        ax.annotate(f'y = {slope:.2f} * x + {intercept:.2f}'+
                    "\n"+f'p_value:{p_value:.2f}' + "\n"+f'r_squared:{r_value ** 2:.2f}',
                    (2, -0.2*np.diff(ax.get_ylim())[0]+ax.get_ylim()[1]))
        ax.set_title('logscale ' + sheet)
        if sheet == "blank_corrected":  
            ax.set_ylabel('log10 wwt')
        else:
            ax.set_ylabel('log10 LW')

    ax.set_xlabel('Carcass Status')


fig.savefig('./pics/pcbs/vsCarcass_status.jpg')

In [ ]:
fish_info = pd.read_excel("fish_info.xlsx")
fish_info['Fish ID'] = fish_info['Fish ID'].astype('str')
fish_info['Carcass Status'] = fish_info['Carcass Status'].astype('int8')
sheets = ("blank_corrected", "lipid_normalized")
for i in range(4):
    fig, ax = plt.subplots(figsize = (6, 5), facecolor = 'white')
    sheet = sheets[i//2]
    pcb = pd.read_excel("./generated excels/pcb.xlsx", sheet_name=sheet)
    c = pcb.columns.tolist()
    c[0] = 'Units'
    pcb.columns = c
    total_pcb = pd.DataFrame(pcb.iloc[-2, :]).reset_index()
    total_pcb.columns = total_pcb.iloc[0, :]
    total_pcb.drop(index = 0, inplace=True)
    total_pcb.iat[-1, 0] = '13'
    total_pcb.columns = ['Fish ID', 'Total PCBs']
    data = pd.merge(fish_info, total_pcb, how='inner', on='Fish ID')
    if i % 2 == 0:
        ax.scatter(*data.loc[:, ['Carcass Status', 'Total PCBs']].to_numpy().T,label = 'points', color = 'k')
        slope, intercept, r_value, p_value, std_err = linregress(*data.loc[:, ['Carcass Status', 'Total PCBs']].to_numpy('float').T)
        x = data.loc[:,'Carcass Status'].to_numpy('float')
        ax.plot(x,slope * x + intercept, label = 'linereg', color = 'green')
        ax.annotate(f'y = {slope:.2f} * x + {intercept:.2f}'+
                    "\n"+f'p_value:{p_value:.2f}' + "\n"+f'r_squared:{r_value ** 2:.2f}',
                    (2, -0.2*np.diff(ax.get_ylim())[0]+ax.get_ylim()[1]))
        ax.set_title(sheet)
        if sheet == "blank_corrected":  
            ax.set_ylabel('wwt')
        else:
            ax.set_ylabel('LW')
    else:
        data['Total PCBs'] = np.log10(data['Total PCBs'].to_numpy('float'))
        ax.scatter(*data.loc[:, ['Carcass Status', 'Total PCBs']].to_numpy().T,label = 'points', color = 'k')
        slope, intercept, r_value, p_value, std_err = linregress(*data.loc[:, ['Carcass Status', 'Total PCBs']].to_numpy('float').T)
        x = data.loc[:,'Carcass Status'].to_numpy('float')
        ax.plot(x,slope * x + intercept, label = 'linereg', color = 'green')
        ax.annotate(f'y = {slope:.2f} * x + {intercept:.2f}'+
                    "\n"+f'p_value:{p_value:.2f}' + "\n"+f'r_squared:{r_value ** 2:.2f}',
                    (2, -0.2*np.diff(ax.get_ylim())[0]+ax.get_ylim()[1]))
        ax.set_title('logscale ' + sheet)
        if sheet == "blank_corrected":  
            ax.set_ylabel('log10 wwt')
        else:
            ax.set_ylabel("log scale $\Sigma PCB \enspace pg.g^{-1} \enspace lw$")

    ax.set_xlabel('Carcass Status')


    fig.savefig(f'./pics/pcbs/vsCarcass_status{i}.jpg') 


In [ ]:
fish_info

In [ ]:
data